# SL Fidelidad - 01_Load Data

Fuentes disponibles:
- caracteristicas
- abonos
- eventos
- aspo

In [ ]:
# !pip3 install openpyxl
# !pip3 install xlrd
!which python

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import pickle

import xlrd
from openpyxl import load_workbook

from helper_functions import str_rem_empty
from helper_functions import info_evento

from functools import reduce

In [2]:
# seteo carpetas
BASE = Path().absolute().parent
RAW = Path(BASE/'raw')
DATA = Path(BASE/'data')

## Carga de datos y preparacion de tablas

#### Tablon de socios

In [3]:
# caracteristicas
df_soc = pd.read_csv(RAW/'padronsocios_202101.csv', sep=';')
df_soc.head()

,Nro soc,Nombre,Activo,Direcci�n,CP,Localidad,Provincia,Tipo,Lista de Precio,Descripcion,...,Fecha Nac.,Alta,Ult. Cuota Paga,Motivo de Baja,Sexo,Relaci�n,Observaciones,CAT,id_tarjeta,Refu
0,1,COLOM MIGUEL,1,BARCO CENTENERA DEL 178 PISO 5 DPTO. B,1424,CAPITAL FEDERAL,CAPITAL FEDERAL,VITALICIO,141,PRECIO ESPECIAL. CATEG: VITALICIO,...,1/1/1950,27/6/1952,1/2/2021,NaN,M,RESPONSABLE DE PAGO,NaN,VITALICIO,0.0,NaN
1,10,DEFINA RAFAEL,1,RIVERA PEDRO IGNACIO DR. 4670,1431,CAPITAL FEDERAL,CAPITAL FEDERAL,VITALICIO,141,PRECIO ESPECIAL. CATEG: VITALICIO,...,1/1/1900,16/7/1954,1/2/2021,NaN,M,RESPONSABLE DE PAGO,NaN,VITALICIO,0.0,NaN
2,100,DETTANO ALFREDO,1,TREINTA Y TRES ORIENTALES 728 PISO 1� DPTO. A,1236,CAPITAL FEDERAL,CAPITAL FEDERAL,VITALICIO,141,PRECIO ESPECIAL. CATEG: VITALICIO,...,14/8/1914,16/5/1963,1/2/2021,NaN,M,RESPONSABLE DE PAGO,NaN,VITALICIO,0.0,NaN
3,1000,GARCIA CARLOS SIMON,1,DRUMOND 2292,1437,CAPITAL FEDERAL,CAPITAL FEDERAL,VITALICIO,141,PRECIO ESPECIAL. CATEG: VITALICIO,...,25/6/1925,26/6/1975,1/2/2021,NaN,M,RESPONSABLE DE PAGO,NaN,VITALICIO,0.0,NaN
4,10001,QUICI MARTA BEATRIZ,1,GREGORIO POMAR 4152 PISO 2 DPTO. B,1437,CAPITAL FEDERAL,BOEDO,ACTIVA PLENA,116,PRECIO ESPECIAL. CATEG: ACTIVA PLENA,...,19/8/1959,16/12/2003,1/2/2021,NaN,F,RESPONSABLE DE PAGO,NaN,PLENO,5.0,NaN


In [ ]:
df_soc.shape

In [ ]:
df_soc.columns

In [ ]:
df_soc['Provincia'].value_counts()

In [ ]:
df_soc['CAT'].value_counts()

In [4]:
# tipo de socio
df_soc['tipo_socio'] = 'SIMPLE'
df_soc.loc[(df_soc['CAT'] == 'INTERIOR'), 'tipo_socio'] = 'INTERIOR'
df_soc.loc[(df_soc['CAT'] == 'EXTERIOR'), 'tipo_socio'] = 'EXTERIOR'
df_soc.loc[(df_soc['CAT'] == 'PATRIM'), 'tipo_socio'] = 'PATRIMONIAL'
df_soc.loc[(df_soc['CAT'] == 'PLENO'), 'tipo_socio'] = 'PLENO'
df_soc.loc[(df_soc['CAT'] == 'VITALICIO'), 'tipo_socio'] = 'VITALICIO'
df_soc['tipo_socio'].value_counts()

SIMPLE         33684
INTERIOR        9394
PLENO           9173
VITALICIO       4093
PATRIMONIAL      718
EXTERIOR         352
Name: tipo_socio, dtype: int64

In [5]:
# forma de pago
df_soc['id_tarjeta'].value_counts()

5.0     29261
0.0     12686
6.0      5831
4.0      5218
1.0      2503
2.0       826
17.0      185
8.0       104
Name: id_tarjeta, dtype: int64

In [6]:
df_soc.loc[(df_soc['id_tarjeta'] != 0), 'forma_pago'] = 'DEBITO'
df_soc.loc[(df_soc['id_tarjeta'] == 0), 'forma_pago'] = 'EFECTIVO'
df_soc['forma_pago'].value_counts()

DEBITO      44728
EFECTIVO    12686
Name: forma_pago, dtype: int64

In [7]:
# refundador
df_soc['Refu'].value_counts()

895.0      4
21150.0    3
23117.0    3
8603.0     2
26909.0    2
          ..
24264.0    1
13192.0    1
24597.0    1
24112.0    1
2150.0     1
Name: Refu, Length: 14281, dtype: int64

In [8]:
df_soc['refunda'] = 0
df_soc.loc[(-df_soc['Refu'].isna()), 'refunda'] = 1
df_soc['refunda'].value_counts()

0    43073
1    14341
Name: refunda, dtype: int64

In [9]:
# aspo
df_soc['Ult. Cuota Paga'].value_counts()

1/2/2021     34496
1/1/2021     15487
1/12/2020     4063
1/11/2020     1875
1/10/2020      910
1/9/2020       555
1/3/2021        15
1/6/2021         4
1/4/2021         3
1/12/2021        2
1/1/2022         1
1/5/2021         1
21/3/2021        1
1/10/2021        1
Name: Ult. Cuota Paga, dtype: int64

In [10]:
df_soc['aspo_dia'] = 0
df_soc.loc[df_soc['Ult. Cuota Paga'].str.slice(start=-4)=='2021', 'aspo_dia'] = 1
df_soc['aspo_dia'].value_counts()

1    50010
0     7404
Name: aspo_dia, dtype: int64

In [11]:
# renombro
df_soc = df_soc.rename(columns={'Alta': 'fecha_ingreso', 'Fecha Nac.': 'fecha_nac', 'Nro soc': 'socio'})

In [12]:
df_socios = df_soc[['socio', 'Sexo', 'tipo_socio', 'forma_pago', 'fecha_ingreso', \
                   'refunda', 'fecha_nac', 'aspo_dia']]
df_socios.columns = [x.lower() for x in df_socios.columns]

In [13]:
df_socios.head()

,socio,sexo,tipo_socio,forma_pago,fecha_ingreso,refunda,fecha_nac,aspo_dia
0,1,M,VITALICIO,EFECTIVO,27/6/1952,0,1/1/1950,1
1,10,M,VITALICIO,EFECTIVO,16/7/1954,0,1/1/1900,1
2,100,M,VITALICIO,EFECTIVO,16/5/1963,0,14/8/1914,1
3,1000,M,VITALICIO,EFECTIVO,26/6/1975,0,25/6/1925,1
4,10001,F,PLENO,DEBITO,16/12/2003,0,19/8/1959,1


In [14]:
df_socios.dtypes

socio             int64
sexo             object
tipo_socio       object
forma_pago       object
fecha_ingreso    object
refunda           int64
fecha_nac        object
aspo_dia          int64
dtype: object

In [15]:
# index a string
df_socios = df_socios.loc[-df_socios['socio'].isna()]
df_socios['socio'] = df_socios['socio'].astype(int).astype(str)
df_socios = df_socios.set_index('socio')

In [16]:
df_socios.head()

,sexo,tipo_socio,forma_pago,fecha_ingreso,refunda,fecha_nac,aspo_dia
socio,,,,,,,
1,M,VITALICIO,EFECTIVO,27/6/1952,0,1/1/1950,1
10,M,VITALICIO,EFECTIVO,16/7/1954,0,1/1/1900,1
100,M,VITALICIO,EFECTIVO,16/5/1963,0,14/8/1914,1
1000,M,VITALICIO,EFECTIVO,26/6/1975,0,25/6/1925,1
10001,F,PLENO,DEBITO,16/12/2003,0,19/8/1959,1


#### Abonados

In [ ]:
# abonos
df_abo = pd.read_csv(RAW/'abonos_historico.csv', sep=';', encoding = "ISO-8859-1")
# df_abo.head()

In [ ]:
# df_abo.shape

In [ ]:
# año del abono
df_abo['abo_year'] = df_abo['Vencimiento'].str.slice(start = -4)
# df_abo['abo_year'].value_counts()

In [ ]:
# cambio de nombre de vars
df_abo = df_abo.rename(columns={'Socio': 'socio', 'Apellido y Nombre': 'nombre'})

In [ ]:
# elimino espacios vacios en ultimo caracter de nombre
df_abo['nombre'] = [str_rem_empty(x) for x in list(df_abo['nombre'])]

In [ ]:
# pivoteo socio y nombre
socio_nombre = df_abo.loc[-df_abo['socio'].isna(), ['socio', 'nombre']]

# elimino la columna socio con NaN
df_abo = df_abo.drop(['socio'], axis=1)

In [ ]:
# merge
df_abo = pd.merge(df_abo, socio_nombre, how='left', on='nombre')

In [ ]:
# filtro abonos que no importan para el analisis
df_abo = df_abo[df_abo['Sector']!='ESTACIONAMIENTO']
# df_abo['Sector'].value_counts()

In [ ]:
# chequeo nulls
print(sum(df_abo['socio'].isna()))
# df_abo.loc[df_abo['socio'].isna(), 'nombre']

In [ ]:
# elimino los nulls que quedaron y casteo a str
df_abo = df_abo.loc[-df_abo['socio'].isna()]
df_abo['socio'] = df_abo['socio'].astype(int).astype(str)

In [ ]:
# df_abo.shape

In [ ]:
# len(df_abo['socio'].unique())

In [ ]:
# genero dataframe con variables por abono
dfs = [
    # bideg 2020
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2020') & (df_abo['Sector'].isin(['PLATEA SUR', 'PLATEA NORTE']))\
                              , 'socio'].unique()
         ,'abo_bid_2020':1}),
    # bideg 2021
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2021') & (df_abo['Sector'].isin(['PLATEA SUR', 'PLATEA NORTE']))\
                              , 'socio'].unique()
         ,'abo_bid_2021':1}),
    # poli 2020
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2020') & (df_abo['Sector']=='POLIDEPORTIVO'), 'socio'].unique()
         ,'abo_pol_2020':1}),
    # poli 2021
    pd.DataFrame(
        {'socio':df_abo.loc[(df_abo['abo_year'] == '2021') & (df_abo['Sector']=='POLIDEPORTIVO'), 'socio'].unique()
         ,'abo_pol_2021':1})
    ]
df_abonos = reduce(lambda left,right: pd.merge(left,right,on=['socio'],
                                            how='outer'), dfs)

In [ ]:
# df_abonos.shape
# df_abonos.head()

In [ ]:
# df_abonos.sum(axis=1).value_counts()

In [ ]:
df_abonos.columns

In [ ]:
df_abonos = df_abonos.set_index('socio')

#### Eventos

In [ ]:
# eventos
eve = load_workbook(filename = RAW/'eventos.xlsx')
eve_sheets = eve.sheetnames

In [ ]:
print(eve_sheets)
eve_sheets.remove('Abonado')
eve_sheets.remove('UNOxSOCIO')

In [ ]:
# ejemplo
# info_evento(eve_sheets[0])

In [ ]:
# corro la funcion para todas las solapas y almaceno los resultados en un diccionario
eventos = {}
for eve in range(len(eve_sheets)):
    eventos[eve] = info_evento(eve_sheets[eve])

In [ ]:
# print(len(eventos))
# print(type(eventos[0]))
# print(eventos[0].keys())

In [ ]:
# publico promedio
np.mean(np.array([int(eventos[x]['publico']) for x in eventos]))

In [ ]:
# socios totales que asistieron a algun evento
socios_eve = []
for eve in eventos:
    socios_eve.extend(list(eventos[eve]['socios']))
# print(len(socios_eve))
socios_eve = list(set(socios_eve))
# print(len(socios_eve))

In [ ]:
# genero un data frame con la # de eventos por socio por año
socios_eventos = pd.DataFrame({'socio': socios_eve
                              ,'eve_2018': np.zeros(len(socios_eve))
                              ,'eve_2019': np.zeros(len(socios_eve))
                              ,'eve_2020': np.zeros(len(socios_eve))})

In [ ]:
# cantidad de partidos asistidos por socio por año
for socio in range(len(socios_eventos['socio'])):
    for evento in eventos:
        if eventos[evento]['fecha'].year == 2018:
            socios_eventos['eve_2018'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])
        if eventos[evento]['fecha'].year == 2019:
            socios_eventos['eve_2019'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])  
        if eventos[evento]['fecha'].year == 2020:
            socios_eventos['eve_2020'][socio] += (socios_eventos['socio'][socio] in eventos[evento]['socios'])

In [ ]:
socios_eventos['socio'] = socios_eventos['socio'].astype(int).astype(str)
df_eventos = socios_eventos.set_index('socio')
# df_eventos.head()

## Salvo las bases

In [17]:
# socios
pickle.dump( df_socios, open( DATA/"df_socios.p", "wb" ) )
# abonos
pickle.dump( df_abonos, open( DATA/"df_abonos.p", "wb" ) )
# eventos
pickle.dump( df_eventos, open( DATA/"df_eventos.p", "wb" ) )